# Correlation Power Analysis (Brier et al. 2004)


#### Learning goals
- Learn the mathematical backgrounds of _Correlation Power Analysis_
- Learn how to perform a Correlation Power Analysis
- Learn how to perform a Correlation Power Analysis using Lascar

#### References
- [E. Brier, C. Clavier, and F. Olivier. Correlation Power Analysis with a Leakage Model. In M. Joye and J.-J. Quisquater, editors, Cryptographic Hardware and Embedded Systems – CHES 2004, volume 3156 of Lecture Notes in Computer Science, pages 16–29. Springer, 2004](https://www.iacr.org/archive/ches2004/31560016/31560016.pdf)
- J. van Woudenberg, C. O'Flynn. The Hardware Hacking Handbook - Chapter "Correlation Power Analysis"

## 1. Introduction

The DPA attack assumes that for a particular device, you'll get a difference in power consumption when a bit is a 1 or a 0.
Any of the 8 bits of one byte is suitable to mount an attack on the AES SBox output.
This redundancy is something we can actually use to strengthen our attack.

The basic of DPA is: "If some intermediate bit varies, the power consumption varies with it."
This does not cover the full extent of the relationship between data and power consumption, which is:

The higher the Hamming weight of a word the higher the power consumption.

Or more precise:

**The power consumption of a device is proportional to the Hamming weight of the data it processes.**

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import lascar
import numpy as np
import plotly.graph_objects as pgo

from securec.capture import capture

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 1</div>

Visualize the statement about linearity (=proportionality):

1. Capture 500 traces with random input in the first byte.
2. Group the traces by the Hamming weight of first input byte.
3. Plot the 500 traces in one graph.
4. Identify (by visual inspection) a sample point where the traces differ.
5. Create a plot "hamming(input) vs. trace[point]". 
   I.e. put the values hw(attempt) on the x-axis and the mean value of the corresponding traces at the proper point on the y-axis.
6. Think about to which instruction(s) the trace-point belongs to.

Hints:
- Use `fromfile=os.path.abspath("../lecture_3/sbox_lookup.c")`.
- If the plot does not look _linear_ look again for a better sweat spot.

## 2. Pearson correlation coefficient

An interesting statistical formula to face this problem is given by the *Pearson correlation coefficient*. For two random variables $X, Y$ it is defined as

$$\rho_{X,Y} := \frac{\mathrm{Cov}(X, Y)}{\sqrt{\mathrm{Var}(X)} \sqrt{\mathrm{Var}(Y)}} \ \in [-1, 1]\,.$$

For two samples of finite length $x = {x_1, ..., x_n}$, $y = {y_1, ..., y_n}$ it can be defined as 

$$r_{x,y} := \frac{\sum_{i=1}^n (x_i - \bar x)(y_i - \bar y)}{\sqrt{\sum_{i=1}^n (x_i - \bar x)^2}\sqrt{\sum_{i=1}^n (y_i - \bar y)^2}} \ \in [-1, 1]\,,$$

where $\bar x := \frac{1}{n} \sum_{i=1}^n x_i$ is the mean of a sample $x$.

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 2</div>

1. Implement Pearson correlation coefficient in python.
2. Test your implementation against the following (x, y)-data with `size = 50`:
    1. `(range(size), 5 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size))` => $r \approx 0.99$
    2. `(range(size), np.array(range(size)) + np.random.uniform(-size, size, size=size) + 10)` => $r \approx 0.32$
    3. `(range(size), -3 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size) + 200)` => $r \approx -0.98$
    4. `(range(size), 100 * np.sin(np.array(range(size)) / size * np.pi + np.pi) + np.random.uniform(-size/10, size/10, size=size))` => $r \approx -0.05$
3. Visualize above example data.

</div>

## 3. Attack!

How to use this as principle for an attack?

We saw that there is a _linear_ relationship between the trace at a given point and the Hamming weight of the input. Now we can quantify a linear dependency!

So, we can develop an attack out of this principle:

1. Record traces with random input.
2. Focus on a point in the program where the input "collides" with the secret. In the case of an AES the output of the SBox lookup is a good point.
3. For each input compute a "forecast" of hamming weights. If we do this for all possible secrets we will see that there is one guess where the Pearson coefficient between the forecast and the traces at this certain point maximizes.

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 3</div>

Implement an automated attack using Correlation Power Analysis, i.e. implement a function with the following signature:

```python
def aes_sbox_cpa(
    traces,
    key_byte_index=0,
    trace_point=42,
):
```

Hints:
- Use a fixed sample point in the traces. We will see later how to get rid of this manual step.
- It is not the Pearson coefficient which is important, but it's _absolute_ value.
- Numpy offers great possibilities to index more dimensional arrays: You can write `data["trace"][:, 42]` to get the 42-th sample point for each recorded trace.

</div>

<div style="border: 3px solid plum; border-radius: 5px; padding: 5px; width: calc(100% - 20px);">
<div class="h2" style="font-variant: all-small-caps;">Exercise 4</div>

Perform a CPA attack using Lascar's `CpaEngine`.

</div>